In [1]:
import pandas as pd

In [8]:
LexRank_data_path = '../resultats/results_LexRank.csv'

df = pd.read_csv(LexRank_data_path)

In [9]:
df.head()

,Text,Reference,Generated
0,"OCTOBER TERM, 2002\n Syllabus\n EARLY, WA...",William Packer was convicted in a California s...,"On direct appeal, the State Court of Appeal re..."
1,"OCTOBER TERM, 2002\n Syllabus\n DOW CHEMI...",In 1984 Dow Chemical Co. negotiated a settleme...,"Syllabus DOW CHEMICAL CO.\nArgued February 26,..."
2,"OCTOBER TERM, 2002\n Syllabus\n SYNGENTA ...",Hurley Henson filed suit in Louisiana state co...,"Argued October 15, 2002-Decided November 5, 20..."
3,OPINION OF THE COURTRUMSFELD V. PADILLA542 U. ...,"Jose Padilla, an American citizen, was arreste...","Padilla’s motion was still pending when, on Ju..."
4,"OCTOBER TERM, 1993\n Syllabus\n CONSOLIDA...",Consolidated Rail Corporation (Conrail) employ...,The injury we contemplate when considering neg...
